In [1]:
import pandas as pd

df = pd.read_csv("ood_travel.csv", delimiter=";").dropna()
df

,text,label
0,Новодевичий монастырь,0
1,Храм Христа Спасителя,0
2,"Тверская, 13",0
3,Сокол,0
4,Зарядье,0
...,...,...
1867,хуястый,1
1868,целка,1
1869,член,1
1870,шмариться,1


In [2]:
import ruclip

device = 'cuda'
clip, processor = ruclip.load('ruclip-vit-base-patch32-384', cache_dir='/tmp/ruclip', device=device)

/home/leffff/PycharmProjects/travel-hack/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/leffff/PycharmProjects/travel-hack/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:669: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(


In [3]:
import torch

@torch.inference_mode()
def get_text_latents(texts):
    text_latents = []

    inputs = processor(text=texts, return_tensors='pt', padding=True)
    text_latents.append(clip.encode_text(inputs['input_ids'].to(device)))

    text_latents.append(torch.cat(text_latents, dim=0))
    text_latents = torch.stack(text_latents).mean(0)

    torch.cuda.empty_cache()
    
    return text_latents

In [4]:
from tqdm import tqdm

embeddings = []
texts = df.text.tolist()

for i in tqdm(range(df.shape[0])):
    embedding = get_text_latents(texts[i]).cpu()
    # print(embedding.device)
    embeddings.append(embedding)
    torch.cuda.empty_cache()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1871/1871 [00:09<00:00, 204.46it/s]


In [5]:
embeddings = torch.cat(embeddings, dim=0).numpy()
embeddings.shape

(1871, 512)

In [7]:
import numpy as np

with open('ood_embeddings.npy', 'wb') as f:
    np.save(f, embeddings)